# Analisis exploratorio de datos (EDA)

El propósito de este apartado es llevar a cabo un profundo análisis de los datos recabados gracias a FinnHub y AlphaVantage, con la finalidad de comprender mejor su naturaleza y relevancia. Esto nos ayudará a poder tomar decisiones informadas a la hora de llevar a cabo el preprocesado y la creación de modelos, teniendo en cuenta cuáles son las variables que más nos interesan o la correlación entre las mismas.

Comenzaremos importando todas las librerías necesarias para realizar esta actividad.

In [34]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

Además, importamos los datos para el estudio

In [5]:
financial_df = pd.read_csv('data/fundamentals.csv', sep=',')

## Feature Analysis

Una vez tenemos cargados los datos procedemos a analizarlos. Primeramente llevamos a cabo un análisis completo de todos los atributos, donde conoceremos los tipos de datos disponibles, sus estadísticas principales y las posibles correlaciones.

CITA: @article{sahoo2019exploratory,
  title={Exploratory data analysis using Python},
  author={Sahoo, Kabita and Samal, Abhaya Kumar and Pramanik, Jitendra and Pani, Subhendu Kumar},
  journal={International Journal of Innovative Technology and Exploring Engineering},
  volume={8},
  number={12},
  pages={4727--4735},
  year={2019}
}

### Data types

En el análisis de los atributos, empezamos comprobando el tipo de los mismos, así como cuantos valores no nulos tenemos.

In [10]:
# Comprobamos los tipos de datos disponibles
print(financial_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64635 entries, 0 to 64634
Data columns (total 74 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Unnamed: 0                                                 64635 non-null  int64  
 1   fiscalDateEnding                                           64635 non-null  object 
 2   totalCurrentAssets                                         45331 non-null  float64
 3   totalAssets                                                55151 non-null  float64
 4   currentAccountsPayable                                     35609 non-null  float64
 5   totalCurrentLiabilities                                    45113 non-null  float64
 6   commonStock                                                50333 non-null  float64
 7   retainedEarnings                                           53105 non-null  float64
 8   totalS

Como era de esperar todas las columnas son float64; exceptuando fiscalDateEnding, reportedCurrency, symbol, sector e industria. Además tenemos una columna de índice "unnamed: 0". Por otro lado, tenemos un total de 64635 filas de información, con 72 "features" o atributos distintos, algunos de ellos contienen muchos null values, por tanto, serán variables que no aportarán casi información. Algunos ejemplos serían shorTermDebt, netIncomeFromcontinuingOperations, entre otros.

Cabe destacar que casi todas las variables tienen algunos valores faltantes, por tanto, se necesitará llevar a cabo una correcta depuración de los datos a través de la eliminación e imputación de atributos.

### Main statistics

A continuación, obtenemos una descripcion de los datos más relevantes, como son los resultados finales de los estados financieros o algunos de los ratios computados

In [24]:
# Obtenemos una descripcion de los datos más relevantes
columns_to_describe = ['totalAssets', 'netIncome', 'totalLiabilities', 'sharePrice', 'totalShareholderEquity', 'changeInCashAndCashEquivalents', 'ROA', 'EPS']
print(financial_df[columns_to_describe].describe())

        totalAssets     netIncome  totalLiabilities    sharePrice  \
count  5.515100e+04  5.083000e+04      6.219800e+04  6.147500e+04   
mean   4.549746e+09  1.159560e+08      3.229495e+09  9.341551e+06   
std    2.881059e+10  1.454275e+09      2.211128e+10  9.515364e+08   
min   -5.921384e+09 -1.250000e+10     -2.465100e+10  3.000000e-04   
25%    7.503469e+07 -1.021925e+07      2.250325e+07  7.410000e+00   
50%    4.026870e+08  2.295000e+05      1.868520e+08  1.690380e+01   
75%    1.806526e+09  1.911600e+07      1.183863e+09  4.025000e+01   
max    4.831333e+12  7.908200e+10      4.132105e+12  1.618176e+11   

       totalShareholderEquity  changeInCashAndCashEquivalents           ROA  \
count            5.188600e+04                    2.328700e+04  5.054700e+04   
mean             1.374187e+09                    2.280225e+07           NaN   
std              8.578987e+09                    8.580501e+08           NaN   
min             -8.768000e+09                   -4.847700e+10 

c:\Users\xavic\Escritorio\TFG_MLStockRating\.venv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\xavic\Escritorio\TFG_MLStockRating\.venv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


A través de estos números no podemos sacar muchas conclusiones, sin embargo, sí podemos detectar ciertas anomalías:
- La media de precios está completamente disparada, con un valor de 9.34e+06. Si nos fijamos en los quartiles 1 y 3 los valores son de 7.41$ y 40.25$ respectivamente, por tanto, una media tan alta nos indica que hay valores muy atípicos que provocan esta gran inflación de los datos.
- Los ratios contienen valores infinitos (ya que puede haber divisiones entre 0, valores nulos, etc.), lo cual dificulta el cálculo de las estadísticas básicas.

Estas observaciones también tendrán que ser tomadas en cuenta en el preprocesado. Por ahora, simplemente quitaremos los valores infinitos en los ratios para poder llevar a cabo un correcto análisis gráfico posteriormente.

In [35]:
# Quitamos valores infinitos
financial_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Obtenemos una descripcion de los ratios
columns_to_describe = ['ROA', 'ROE', 'EPS', 'P/E', 'bookValue', 'currentRatio', 'totalAssetsTurnover', 'inventoryTurnover']
print(financial_df[columns_to_describe].describe())

                ROA           ROE           EPS           P/E     bookValue  currentRatio  totalAssetsTurnover  inventoryTurnover
count  5.052700e+04  4.833500e+04  4.510400e+04  4.451700e+04  4.753400e+04  44817.000000         10535.000000       10862.000000
mean  -4.197336e+02 -4.329519e+02 -2.238940e+02 -2.481297e+05  5.850364e+04      5.055005            -5.123162           7.381662
std    1.064813e+05  1.088656e+05  2.152570e+05  4.549118e+07  2.551156e+06     49.585469           103.311548          56.359526
min   -2.380400e+07 -2.380400e+07 -2.380400e+07 -9.567484e+09 -1.815542e+07      0.000000         -5532.911528          -0.105596
25%   -1.202546e-01 -1.595883e-01 -3.399066e+02 -2.994771e-02  3.048654e+01      1.372601            -0.160475           0.934918
50%    1.067135e-03  1.463818e-02 -2.737710e-03  0.000000e+00  9.612859e+02      2.448143             0.014242           1.866852
75%    1.846662e-02  7.113904e-02  4.743600e+01  4.759223e-01  4.676189e+03      5.038463 

Como se puede ver, en este caso, a pesar de haber eliminado valores infinitos seguimos teniendo valores excesivamente atípicos como máximos y mínimos que provocan que las medias se alejen en muchos casos de los valores interquartílicos. Por tanto, en los ratios también tendremos que limpiar los "outliers". 

Por otro lado, también debemos estudiar los sectores e industrias presentes en los datos, que al ser datos categóricos no aparecen en el análisis anterior.

In [45]:
# Visualización de la distribución de sectores e industrias
sector = financial_df['sector'].value_counts(normalize=True)
print(sector,end='\n\n')

industria = financial_df['industria'].value_counts(normalize=True)
print(industria.head())

# Medias sobre los sectores
promedios = financial_df.groupby('sector')[['ROE', 'P/E', 'sharePrice']].mean()
print(promedios)


sector
LIFE SCIENCES                 0.282566
FINANCE                       0.168709
MANUFACTURING                 0.158784
TRADE & SERVICES              0.145276
TECHNOLOGY                    0.130373
REAL ESTATE & CONSTRUCTION    0.066575
ENERGY & TRANSPORTATION       0.047717
Name: proportion, dtype: float64

industria
PHARMACEUTICAL PREPARATIONS                        0.129148
STATE COMMERCIAL BANKS                             0.068203
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)    0.037265
SERVICES-PREPACKAGED SOFTWARE                      0.037079
SURGICAL & MEDICAL INSTRUMENTS & APPARATUS         0.033729
Name: proportion, dtype: float64
                                    ROE            P/E    sharePrice
sector                                                              
ENERGY & TRANSPORTATION        0.127194      18.064097  4.680242e+03
FINANCE                    -2959.905402      52.254881  2.899771e+01
LIFE SCIENCES                168.246451 -808307.688132  3.305434e+0

Como puede observarse, contamos con información sobre 7 sectores distintos, predominando la información sobre empresas del sector de las ciencias biológicas, destacando las compañías farmacéuticas entre las industrias más presentes en la base de datos.

Por otro lado, puede comprobarse con la tercera tabla la disparidad de datos entre diferentes sectores. Sin embargo, debemos tener en cuenta que estos datos están posiblemente distorsionados a causa de valores atípicos.

### Correlations

Ahora, estudiaremos la correlación de las diferentes variables numéricas. Debido a que la matriz es muy grande, vamos a centrarnos en aquellas variables con relaciones más fuertes (>0.9 ó <-0.9):

In [36]:
# Matriz de correlacion
numerical_variables = financial_df.select_dtypes(include=['float64']).columns
corr_matrix = financial_df[numerical_variables].corr()

# Filtrar las correlaciones fuertes, sin tener en cuenta las diagonales
strong_correlations = (corr_matrix.abs() > 0.9) & (corr_matrix != 1.0)

# Encontrar pares con correlación fuerte
strong_pairs = [(i, j) for i in numerical_variables for j in numerical_variables if strong_correlations.loc[i, j]]

# Crear un DataFrame con los pares y sus valores de correlación
corr_values = [corr_matrix.loc[pair[0], pair[1]] for pair in strong_pairs]
strong_corr_df = pd.DataFrame(strong_pairs, columns=['Variable 1', 'Variable 2'])
strong_corr_df['Correlation'] = corr_values


# Ajustar opciones de visualización
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 1000)       # Ajustar el ancho para la visualización del DataFrame
print(strong_corr_df)

                              Variable 1                            Variable 2  Correlation
0                            totalAssets                      totalLiabilities     0.973980
1                            totalAssets                 totalNonCurrentAssets     0.955311
2                            totalAssets                shortLongTermDebtTotal     0.926476
3                 currentAccountsPayable               otherCurrentLiabilities     0.908467
4                      operatingExpenses                researchAndDevelopment     0.933739
5                        operatingIncome                       incomeBeforeTax     0.994594
6                        operatingIncome                             netIncome     0.983030
7                        operatingIncome                     operatingCashflow     0.967237
8                        operatingIncome           comprehensiveIncomeNetOfTax     0.905957
9                        operatingIncome                                  ebit  

Como podemos ver hay una gran cantidad de correlaciones fuertes, sin embargo, en datos financieros esto es normal ya que muchas variables provienen del cómputo de otras. Por ejemplo, la relación entre P/E y el precio de la acción (P/E - sharePrice), o el beneficio bruto y el beneficio neto (grossProfit - netIncome). Por tanto, esto no debería suponer un problema.

## Graphical Analysis

A continuación llevaremos a cabo la representación de numerosos gráficos para poder entender los datos de una manera más interactiva y visual

### Análisis de distribuciones

En este apartado comprobaremos las distribuciones de los datos para conocer sus dominios, sesgos y valores atípicos.

Graficos de barras, histogramas y graficos de densidad

### Análisis de series temporales

A través de gráficos temporales podremos observar las tendencias y la evolución histórica de diferentes atributos. Lo que nos ayudará a entender mejor el contexto económico.

Gráficos de líneas y área

### Análisis multivariable

En este apartado nos centramos en entender mejor la relación entre variables.

Gráficos radar y mosaico